# <center> TP4 - Chiffrement asymétrique : RSA </center>
<center> 2023/2024 - L. Naert, T. Ferragut, T. Godin </center>

_Certains exemples et textes de ce TP sont tirés de Exercices et Problèmes de cryptographie de Damien Vergnaud, 3ème édition ainsi que de Codes et Cryptologie de Christine Bachoc._

In [3]:
import numpy as np
import math
import random
import time

Jusqu'à maintenant, nous n'avions vu en TP que des méthodes de cryptographie symétrique dans lesquelles la clef de chiffrement sert également de clef de déchiffrement et doit donc rester secrète pour pouvoir garantir la confidentialité des messages. Vous avez pu, au TP3, transmettre un message chiffré à votre voisin(e). Pour que celui/celle-ci puisse le déchiffrer, vous avez également dû lui fournir la clef de chiffrement ou, dans le cas plus précis du LFSR, les informations permettant à votre voisin(e) de calculer la clef de chiffrement. C'est ce que l'on appelle l'_échange des clefs_. Il s'agit d'un problème crucial dès que l'on utilise une technique de chiffrement symétrique. 
Si Alice veut envoyer un message à Bob sans que celui-ci puisse être lu par Oscar, elle ne peut pas envoyer à la fois le message et la clef puisque, si Oscar intercepte ces informations, il sera en mesure de lire le message. Il est donc necessaire de définir un protocole d'échange de clef.

La cryptographie asymétrique, aussi appelé cryptographie à clef publique (à opposer à la cryptographie symétrique/à clef secrete), permet de résoudre la problématique le l'échange de clef.

## 1 - Principe de la cryptographie asymétrique

Dans un système de cryptographie asymétrique, les interlocuteurs possèdent __chacun__ une clef composée de deux parties : une partie publique que tout le monde est en mesure de voir et une partie privée que seul le propriétaire de la clef connait. Ainsi, si l'on considère deux protagonistes Alice et Bob, Alice possède une clef $k_A = (k_A^{pub},k_A^{priv})$ et Bob possède une autre clef $k_B = (k_B^{pub},k_B^{priv})$

La fonction de chiffrement $E$ est paramétrée par la partie publique de la clef du destinataire tandis que la fonction de déchiffrement $D$ est paramétrée par la partie privée de la clef du destinataire. 

Ainsi, si Alice souhaite envoyer un message à Bob, elle chiffre son message $m$ en utilisant la partie publique de la clef de Bob (aucun problème puisque tout le monde à accès aux clefs publiques) :
$$ c = E_{k_B^{pub}}(m) $$

Bob déchiffrera le chiffré $c$ en utilisant la partie privée de sa clef (qu'il est le seul à connaitre) :
$$ m = D_{k_B^{priv}}(c)$$


Ainsi tout le monde peut écrire des messages à Bob en utilisant la clef publique de Bob mais seul Bob peut les déchiffrer grâce à sa clef privée. (Par abus de langage, on dit souvent "clef privée (resp. publique)" au lieu de "la partie privée (resp. publique) de la clef")

Pour que ce système fonctionne, il faut que __la partie publique et la partie privée de la clef permettent de définir des opérations $E$ et $D$ réciproques ($ D_{k^{priv}} = E_{k^{pub}}^{-1}$) et que le calcul de la clef privée de quelqu'un connaissant sa clef publique soit impossible__ (ou, plus justement, infaisable dans des temps raisonnables).

Ce principe de la cryptographie asymétrique a été formalisé par Diffie et Hellmann en 1976 mais aucune solution concrète n'avait été proposée à ce moment. Il a fallu attendre le chiffrement RSA, proposé par Rivest, Shamir et Adleman un an plus tard pour pouvoir implémenter le chiffrement asymétrique. 

## 2 - RSA

RSA propose une application concrète du principe du chiffrement asymétrique en se basant sur la difficulté à factoriser des entiers de grande taille.

Une clef RSA $k = (k^{pub},k^{priv})$ est définie à partie des paramètres suivants : 
- $p$ et $q$ sont deux grands nombres premiers distincts
- $N = pq$
- $e$ et $d$ sont des entiers tels que $ed = 1 \mod \varphi(N)$ ($d$ est l'inverse de $e$ modulo $\varphi(N)$)


Alors $k^{pub} = (N,e)$ et $k^{priv} = (N,d)$

### a) Indicatrice d'Euler

[_Définition_](https://fr.wikipedia.org/wiki/Indicatrice_d%27Euler) : 
On appelle __indicatrice d'Euler__, notée $\varphi$, la fonction, qui à tout entier naturel $n$ non nul associe le nombre d'entiers compris entre $1$ et $n$ (inclus) et premiers avec n: 

\begin{array}{ccccl}\varphi &:&\mathbb {N} ^{*}&\longrightarrow &\mathbb {N} ^{*}\\&&n&\longmapsto &\mathrm {card} (\{m\in \mathbb {N} ^{*}~|~m\leqslant n~{\text{et}}~pgcd (m,n) = 1\}).\end{array}

> __Question 1 (indicatrice d'Euler)__ : Calculer à la main $\varphi(1)$, $\varphi(4)$ et $\varphi(11)$ et complétez la partie "tests" de la cellule ci-dessous avec vos résultats. Ecrire une fonction `phi(n)` qui calcule l'indicatrice d'Euler d'un entier `n` $\in \mathbb {N}^{*}$. A quoi est égale l'indicatrice d'Euler d'un nombre premier ?

Note : A tout moment, vous pouvez réutiliser des fonctions écrites dans les TP précédents.

In [4]:
def estInversible(a,n):
    while n != 0 : 
        a, n = n, a % n
    return a == 1


def inverse(a, n) :
    x = 0
    if estInversible(a,n) : 
        while (a*x)%n != 1 : 
            x += 1
        return x
    else : 
        return -1
    
def listeInversibles(n):
    lst = []
    for k in range(n) : 
        if estInversible(k,n) : 
            lst.append(k)
    return lst


def phi(n):
    return len(listeInversibles(n))

# Tests
try:
    assert phi(1) ==  1
    assert phi(4) ==  2
    assert phi(11) ==  10
    assert phi(21) == 12 # Premiers avec 21 : [1, 2, 4, 5, 8, 10, 11, 13, 16, 17, 19, 20]
    assert phi(30) == 8 # Premiers avec 30 : [1, 7, 11, 13, 17, 19, 23, 29]
    assert phi(31) == 30 # Premiers avec 31 : Nombres de 1 à 30
    print("phi : OK")
except:
    print("phi : ERREUR")

phi : OK


__Propriété :__ 
Pour tout $u$, $v \in \mathbb {N}^{*} $  tels que $u$ et $v$ sont premiers entre eux (i.e. $pgcd(u,v) = 1$),  $\varphi (uv) = \varphi (u)\varphi (v)$

> __Question 2 (indicatrice d'Euler pour la multiplication de nombres premiers)__ : A quoi est égal $\varphi (uv)$ avec $u$ et $v$ premiers ? Coder la fonction `phiPremier(u,v)` qui calcule $\varphi (uv)$ avec $u$ et $v$ premiers



Solution : 

In [5]:
def phiPremier(u,v):
    '''
    u et v sont des nombres premiers
    '''
    return (u-1) * (v-1)

try:
    assert phiPremier(11,31) ==  300
    assert phiPremier(17,11) ==  phi(17*11)
    print("phiPremier : OK")
except:
    print("phiPremier : ERREUR")


phiPremier : OK


### b) Clefs RSA

> __Question 3 (Exemple RSA)__ : Prenons $p = 11$, $q = 17$ et $e = 7$. Calculer $k^{pub}$ et $k^{priv}$.

In [6]:
p = 11
q = 17
N = p*q
e = 7
d = inverse(e,phi(N))
print("kpub = (" + str(N) + "," + str(e) + ")" )
print("kpriv = (" + str(N) + "," + str(d) + ")")



kpub = (187,7)
kpriv = (187,23)


> __Question 4 (Validité)__ : 
- Ecrire une fonction `estPremier(n)` qui vérifie que n (entier strictement positif) est un nombre premier.
- Ecrire une fonction `estValide(p,q)` qui vérifie que le couple (p,q) est un couple d'entiers valide pour le chiffrement RSA.
- Ecrire une fonction `choixE(p,q)` qui propose une valeur de e pertinente

In [7]:
def estPremier(n):
    '''
    n est un entier strictement positif
    '''
    if n <= 1 : 
        return False
    for i in range(int(math.sqrt(n))) : 
        if (i > 1):
            if n % i == 0 :
                return False
    return True

try:
    assert estPremier(1) ==  False
    assert estPremier(2) ==  True
    assert estPremier(11) ==  True
    assert estPremier(17) ==  True
    assert estPremier(21) ==  False
    print("estPremier : OK")
except:
    print("estPremier : ERREUR")

estPremier : OK


In [8]:
# P et Q premiers 
# P et Q différents
def estValide(p,q):
    if p == q:
        return False
    p_valide = estPremier(p)
    q_valide = estPremier(q)
    return p_valide and q_valide

try:
    assert estValide(11,17) ==  True
    assert estValide(11,11) ==  False
    assert estValide(11,21) ==  False
    print("estValide : OK")
except:
    print("estValide : ERREUR")

estValide : OK


In [9]:
import random

# E doit être inversible modulo phi(n)
# E différent de 1
def choixE(p,q):
    n = p*q
    phi_n = phi(n)
    e = random.randint(0,phi_n) %phi_n
    while not estInversible(e,phi(n)) :
        e = random.randint(0,phi_n)%phi_n
    return e
    

try:
    for k in range(100):
        assert choixE(11,17) in listeInversibles(phiPremier(11,17))
    print("choixE : OK")
except:
    print("choixE : ERREUR")

choixE : OK


> __Question 5 (RSA)__ : Ecrire une fonction `genPubPriv(p,q)` qui, si le couple (p,q) est valide, renvoie une proposition de clef publique/clef privée.

In [33]:
def genPubPriv(p,q):
    if not estValide(p,q) : 
        return False
    n = p*q
    e = choixE(p,q)
    d = inverse(e,phi(n))%phi(n)
    return ((n,e),(n,d))

print(genPubPriv(11,17))#pourrait renvoyer ([187, 29], [187, 149]) par exemple
print(genPubPriv(5407,5413))

((187, 23), (187, 7))
((29268091, 13631689), (29268091, 1641385))


Un attaquant, pour déterminer $k^{priv}$ aura accès à $k^{pub}$, c'est à dire à $N$ et $e$. Or, pour calculer le $d$ de $k^{priv}$, il aura besoin de calculer $\varphi(N)$ : soit en trouvant les deux facteurs premiers $p$ et $q$ de $N$, soit en comptant le nombre de nombres premiers avec $N$. Ces opérations sont difficiles voire impossibles à réaliser en temps raisonnable par des méthodes connues actuellement pour des grands entiers (longueur de $p$ et $q$ supérieure à 512 bits). C'est là où réside la force du chiffrement asymétrique RSA.

### c) Chiffrement/Déchiffrement RSA


Voici les fonctions de chiffrement $E_{k^{pub}}$ et déchiffrement $D_{k^{priv}}$ pour RSA. Remarquez que les messages à chiffrer/déchiffrer sont des éléments de $\mathbb{Z}/N\mathbb{Z}$ avec $N=pq$

\begin{align*}
  E_{k^{pub}} \colon \mathbb{Z}/N\mathbb{Z} &\to \mathbb{Z}/N\mathbb{Z}\\
  m &\mapsto c = m^e
\end{align*}



\begin{align*}
  D_{k^{priv}} \colon \mathbb{Z}/N\mathbb{Z} &\to \mathbb{Z}/N\mathbb{Z}\\
  c &\mapsto m = c^d
\end{align*}


> __Question 6 (Chiffrement)__ : Ecrire une fonction `chiffrementRSA(msgClair,k_pub)` qui effectue le chiffrement de `msgClair`(de type int) à l'aide de la fonction de chiffrement et de la clef publique donnée (`k_pub` est une liste à deux éléments $N$ et $e$).



In [11]:
def chiffrementRSA(msgClair,k_pub):
    return msgClair**k_pub[1]%k_pub[0]

try:
    assert chiffrementRSA(9,[143, 7]) == 48
    assert chiffrementRSA(89,[143, 7]) == 67
    assert chiffrementRSA(89,[187, 119]) == 166
    print("chiffrementRSA : OK")
except:
    print("chiffrementRSA : ERREUR")

chiffrementRSA : OK


> __Question 7 (Déchiffrement)__ : Ecrire une fonction `dechiffrementRSA(msgChiffre,k_priv)` qui effectue le déchiffrement de `msgChiffre`(de type int) à l'aide de la fonction de déchiffrement et de la clef privee donnée (`k_priv` est une liste à deux éléments $N$ et $d$).

In [12]:
def dechiffrementRSA(msgChiffre,k_priv):
    return msgChiffre**k_priv[1]%k_priv[0]

try:
    assert dechiffrementRSA(48,[143,103]) == 9
    assert dechiffrementRSA(80,[187, 109]) == 48
    print("dechiffrementRSA : OK")
except:
    print("dechiffrementRSA : ERREUR")
    

dechiffrementRSA : OK


Encore une fois, on peut remarquer que l'attaquant a besoin du $d$ de $k^{priv}$ pour déchiffrer le message. Cette clef ne peut être calculée qu'en factorisant $N$ en ses deux facteurs premiers $p$ et $q$ (ou en calculant directement $\varphi(N)$, opération tout aussi compliquée pour des grands nombres $N$). En pratique les nombres premiers $p$ et $q$ doivent être très grands pour que la factorisation de $N$ soit une opération impossible en temps raisonnable. 


> __Question 8 (p et q grands)__ : Tester la génération de clefs publique et privée, le chiffrement et le déchiffrement d'un message avec des valeurs de $p$ et $q$ plus importantes (à 4 chiffres par exemple). Que remarquez-vous ?

In [13]:
import time
start = time.time()
pub_key,priv_key = genPubPriv(1381,1399)
message = 125832
print(f"Message : {message}")
cipher = chiffrementRSA(message,pub_key)
print(f"Message chiffré : {cipher}")
message = dechiffrementRSA(cipher,priv_key)
print(f"Message déchiffré : {message}")
end = time.time()
print(f"Paire de clés trouvée en : {round(end-start,3)}s")


Message : 125832
Message chiffré : 1273443
Message déchiffré : 125832
Paire de clés trouvée en : 19.26s


## 3 - Application du chiffrement RSA à l'échange de clef

Le chiffrement RSA, et les chiffrements asymétriques en général, permettent de résoudre le problème de l'échange des clefs du chiffrement symétrique puisque plus aucun échange n'est nécessaire. Cependant, ces types de chiffrement ne peuvent pas se substituer complètement aux chiffrements symétriques car les opérations de chiffrement et déchiffrement prennent trop de temps pour être utilisées régulièrement pour coder de longs messages. Les chiffrements symétriques et asymétriques sont donc souvent tous les deux utilisés pour chiffrer un message : 
- Le chiffrement asymétrique sert à l'échange de clef secrète : il permet de transmettre la clef secrète du chiffrement symétrique de manière sécurisée
- Le chiffrement symétrique est utilisé pour le chiffrement du message lui-même.

> __Activité (synthèse)__ : 
1. Chiffrez un message en utilisant un masque jetable généré de façon pseudo-aléatoire (cf. TP3) et envoyez à votre voisin(e) le message chiffré. 
2. Demander ensuite à votre voisin(e) de générer sa clef RSA (à partir de nombres premiers de 3 chiffres) et de vous transmettre la partie publique de cette clef. Servez-vous de celle-ci pour lui transmettre de manière confidentielle la clef secrete de votre masque.
3. Votre voisin(e) arrive t-il à déchiffrer votre message ?

Chaque étudiant doit réaliser un chiffrement et un déchiffrement !

In [14]:
lfsr_cipher = 11011010011011000011000001111111001000100100111001110110001110101010100101100111011010100110011110000100

In [29]:
pub_key,priv_key = ((29268091, 13631689), (29268091, 1641385))
# Chiffrement Alan
message = "Mais dis moi UTT"
graine = 11011
coeff = 6023
# Pub key alan
pub_key = [38021, 19181]

# Chiffrement des infos
coeff_cipher = chiffrementRSA(coeff,pub_key)
graine_cipher = chiffrementRSA(graine,pub_key)

print(coeff_cipher,graine_cipher)
# 1886 11234

1886 11234


In [19]:
# Alan
coeff_cipher = 133 
graine_cipher = 118
cipher = "0011100110001101000000011001110010010100111101000101110000100101110010010000011010001010"
priv_key = (187,117)

coeff = dechiffrementRSA(coeff_cipher,priv_key)
graine = dechiffrementRSA(graine_cipher,priv_key)
print(graine,coeff)
# dechiffrementLFSR("0011100110001101000000011001110010010100111101000101110000100101110010010000011010001010",[0,1,0,1], [1,2])
message = 'e4sy_crypto'

101 12


In [20]:
# Youenn
graine_cipher = 91
coeff_cipher = 133
# taille + coeff = 17

priv_key = (187,117)
coeff = dechiffrementRSA(coeff_cipher,priv_key)
graine = dechiffrementRSA(graine_cipher,priv_key)
print(graine,coeff)


75 12


_Remarque_ : le module existant Crypto.RSA de Python permet de générer des clefs d'une taille spécifiée, de chiffrer et déchiffrer des messages de manière plus efficace. 

(!pip install crypto)